# LSTM Single Model

---

In [ ]:
# DataFrame
import pandas as pd
import numpy as np
import random

# Date
from datetime import datetime, date
import matplotlib.dates as mdates

# Preprocessing
from sklearn.preprocessing import MinMaxScaler

# Visualization
import matplotlib.pyplot as plt
import matplotlib

# Save the log
import os
import time

# LSTM
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.models import Sequential

# Optimization
from keras_tuner.tuners import RandomSearch

import tempfile

# Metric 
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import root_mean_squared_error
from sklearn.metrics import r2_score

In [ ]:
# set the seed
def set_seed(seed: int):
    random.seed(seed)
    np.random.seed(seed)
    os.environ["PYTHONHASHSEED"] = str(seed)
    tf.random.set_seed(seed)

In [ ]:
# Minus
matplotlib.rcParams['axes.unicode_minus'] = False
# font
plt.rcParams['font.family'] = 'Serif'

In [ ]:
def split_data(product_df, time_steps): 
    
    train_end = len(product_df[product_df['Date']<'2022-07-01'])
    
    features = product_df.drop(['Date','Product', '년월'], axis=1).columns.tolist()
    
    global n_features
    n_features = len(features)
    target_idx = features.index('y')
    
    filtered_df = product_df.filter(features)  
    sc = MinMaxScaler() 
    
    y_train_scaled = sc.fit_transform(filtered_df.iloc[:train_end, :])
    
    X_train = [] 
    y_train = []
    for i in range(time_steps, train_end):
        X_train.append(y_train_scaled[i-time_steps:i, :])  
        y_train.append(y_train_scaled[i, target_idx])  
        
    X_train, y_train = np.array(X_train), np.array(y_train)
    X_train = np.reshape(X_train, (X_train.shape[0], X_train.shape[1], n_features))  
    
    y_test_scaled = sc.transform(filtered_df.iloc[train_end:, :])
    
    X_test = []
    y_test = product_df.iloc[train_end+time_steps:].copy()
    y_test['y_norm'] = y_test_scaled[time_steps:, target_idx]  
    
    for i in range(time_steps, len(y_test_scaled)):
        X_test.append(y_test_scaled[i-time_steps:i, :])
    
    X_test = np.array(X_test)
    X_test = np.reshape(X_test, (X_test.shape[0], X_test.shape[1], n_features))

    return X_train, y_train, X_test, y_test, sc, target_idx

In [ ]:
def build_model(hp):
    model = Sequential()

    model.add(LSTM(units=hp.Int('units_1', min_value=256, max_value=320, step=64),
                   activation='tanh',
                   return_sequences=True, 
                   input_shape=(None, n_features)))
    
    model.add(LSTM(units=hp.Int('units_2', min_value=128, max_value=256, step=32),
                   activation='tanh',
                   return_sequences=False))

    model.add(Dense(units=hp.Int('dense_unit', min_value=16, max_value=128, step=16),
                    activation='tanh'))
        
    model.add(Dense(1))

    model.compile(optimizer=keras.optimizers.Adam(hp.Choice('learning_rate', [1e-2, 1e-3, 1e-4])),
                  loss='mean_squared_error',
                  metrics=['mse'])

    return model

def optimize_model(X_train, y_train, X_test, sc, epochs, trials, target_idx):
    
    with tempfile.TemporaryDirectory() as temp_dir:
        tuner = RandomSearch(
            build_model,
            objective='loss',
            max_trials= trials,
            directory=temp_dir,
            project_name='temp_project')

    tuner.search_space_summary()

    tuner.search(X_train, 
                 y_train,
                 epochs=epochs,
                 batch_size=8)

    tuner.results_summary()

    best_model = tuner.get_best_models(num_models=1)[0]

    pred = best_model.predict(X_test) 
    pred_norm = pred 
    
    pred_expanded = np.zeros((pred.shape[0], n_features))
    pred_expanded[:,target_idx] = pred.ravel()  
    
    pred = sc.inverse_transform(pred_expanded)
    pred = pred[:, target_idx]  

    best_model.summary()

    return best_model, pred, pred_norm

### LSTM Single Model

In [ ]:
def save_model(best_model, product_code):
    path = f'./Result/LSTM/Model/{product_code}.h5'
    best_model.save(path)
    return 

def use_model(X_test, product_code, target_idx, sc):
    path = f'./Result/LSTM/Model/{product_code}.h5'
    best_model = tf.keras.models.load_model(path)
    
    pred = best_model.predict(X_test)
    pred_norm = pred 
    
    pred_expanded = np.zeros((pred.shape[0], n_features))
    pred_expanded[:,target_idx] = pred.ravel()  
    
    pred = sc.inverse_transform(pred_expanded)
    pred = pred[:, target_idx]  
    
    best_model.summary()

    return best_model, pred, pred_norm

In [ ]:
def LSTM_single(product_df, time_steps, epochs, trials, saved_model: bool):
    product_code = product_df['Product'].unique()[0]
   
    X_train, y_train, X_test, y_test, sc, target_idx = split_data(product_df, time_steps)
    
    # use the existing model
    if saved_model:
        best_model, pred, pred_norm = use_model(X_test, product_code, target_idx, sc)
    # save the new model
    else:
        best_model, pred, pred_norm = optimize_model(X_train, y_train, X_test, sc, epochs, trials, target_idx)
        save_model(best_model, product_code)


    y_test.reset_index(drop=True, inplace=True)
    pred_df = pd.DataFrame({'Pred': pred.reshape(-1) ,'Pred_norm': pred_norm.reshape(-1)})
    res_df = pd.concat([y_test, pred_df], axis=1)
    res_df.set_index('Date', inplace=True)
    res_df.loc[res_df['Pred']<0, 'Pred']=0
    # res_df: ['y', 'y_norm', 'Pred', 'Pred_norm'], index='Date'
        
    return res_df

In [ ]:
def actual_pred_plot(product_code, res_df, metric_df):
    today = date.today()
    """
    Plot the actual vs predition and save the figure in the given directory
    """
    res_df.index = pd.to_datetime(res_df.index)
    save_path = os.path.join("Result", "LSTM", product_code)
    
    title = f"Pred Actual Plot - {product_code}"
    actual = res_df['y']
    pred = res_df['Pred']

    # Plot   
    plt.figure(figsize=(16, 8))
    plt.title(title, fontsize=20)
    plt.xlabel("Date", fontsize=14)
    plt.ylabel("Order Demand", fontsize=14)

    plt.plot(res_df.index, actual, label ='Actual', color='r')
    plt.plot(res_df.index, pred, label='Prediction', color='b')
    
    plt.gca().xaxis.set_major_locator(mdates.MonthLocator())
    plt.gca().xaxis.set_major_formatter(mdates.DateFormatter('%Y-%m'))
    plt.legend(loc="upper right")
    
    if not os.path.exists(save_path):
        os.makedirs(save_path)
    # save the figure
    plt.savefig(os.path.join(save_path, product_code+'.png'))
    plt.show()
    
    metric_df.to_csv(os.path.join(save_path, product_code+'_metric.csv'), encoding="utf-8-sig")
    res_df.to_csv(os.path.join(save_path, f'{product_code}_total_result.csv'), encoding="utf-8-sig")    
    plt.close('all') # close all figures to free up memory

In [ ]:
def mape(actual, pred): 
    actual, pred = np.array(actual), np.array(pred)
    return np.mean(np.abs((actual - pred) / (actual+1)))

def nrmse(y_true, y_pred):
    mse = root_mean_squared_error(y_true, y_pred)
    target_mean = np.mean(y_true)
    nrmse = mse / target_mean
    return nrmse

def nmae(y_true, y_pred):
    mae = mean_absolute_error(y_true, y_pred)
    target_mean = np.mean(y_true)
    nmae = mae / target_mean
    return nmae

In [ ]:
def calculate_metrics(product_code, res_df):
  
    actual = res_df['y']
    pred = res_df['Pred']

    MAPE = mape(actual, pred) 
    RMSE = root_mean_squared_error(actual, pred)
    MAE = mean_absolute_error(actual,pred) 
    NRMSE = nrmse(actual,pred) 
    NMAE = nmae(actual,pred)
    R2 = r2_score(actual,pred) 

    metric_df = pd.DataFrame({'MAPE':[round(MAPE, 4)],
                           'RMSE':[round(RMSE, 4)],
                           'MAE':[round(MAE, 4)],
                           'NRMSE':[round(NRMSE, 4)],
                           'NMAE':[round(NMAE, 4)],
                           'R2':[round(R2, 4)]},
                            index= [product_code])

    return metric_df

In [ ]:
def execute_single_LSTM(product_code, time_steps=30, epochs=50, trials=5):
    start_time = time.time()

    product_df = df[df['Product']== product_code].reset_index(drop=True)
    
    res_df = LSTM_single(product_df, time_steps, epochs, trials, saved_model=True) #dictionary, time_steps, epochs
    
    metric_df= calculate_metrics(product_code, res_df)

    actual_pred_plot(product_code, res_df, metric_df)

    elapsed_time_seconds = time.time() - start_time
    elapsed_time_minutes = elapsed_time_seconds / 60
    print("실행 시간: {:.2f} 분".format(elapsed_time_minutes))
    return metric_df

---

In [ ]:
df = pd.read_csv("../Data/dataset.csv")
df['Date'] = pd.to_datetime(df['Date'])

In [ ]:
all_metric = pd.DataFrame()

target_code = ['Office Product', 'Packaging material', 'Pharmaceuticals']
for code in target_code:
    print("==================================")
    print(f"========== { code } ==========")
    print("==================================")
    tmp_metric = execute_single_LSTM(code)
    all_metric = pd.concat([all_metric, tmp_metric])